In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
df = pd.read_csv('data/8-gon-gh-dataset.csv')

good_data = pd.isnull(df['dum1'])

dataset = df[good_data].dropna(axis=1, how='all')

# Split dataset into 70/15/15 training/validation/test
train = dataset.sample(frac=0.85)
test = dataset.drop(train.index)

# Split dataset into features and labels; last 4 (grid scores)
train_features = train.iloc[:, 1:]
train_labels = train.iloc[:, :1]

test_features = test.iloc[:, 1:]
test_labels = test.iloc[:, :1]

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,x6,y6,x7,y7,ix0,iy0,ix1,iy1,ix2,iy2
6,1.119785,0.039213,0.332836,0.544491,0.213457,1.025150,-0.627492,0.987861,-0.688993,0.155813,...,0.262866,-1.423671,0.590690,-0.201720,-0.067951,-0.166750,-0.170636,0.513827,0.681104,0.127328
18,0.915908,0.596912,0.923602,0.918246,0.385590,1.001687,-0.215460,1.045480,-0.622458,0.215055,...,0.083952,-1.376717,0.682998,-0.082009,0.020068,0.519190,0.477077,0.379075,0.333169,-0.266751
43,0.569095,-0.069833,0.540902,0.789540,-0.305556,1.099521,-0.724683,1.202892,-0.903440,-0.083918,...,-0.232435,-1.069934,0.810941,-0.581951,-0.510538,0.435534,-0.048890,0.367831,0.152312,-0.534331
44,0.895173,0.048712,0.665237,0.560939,0.047844,0.791258,-0.853415,1.197531,-0.898977,-0.162231,...,-0.130734,-0.771453,0.424269,-0.463431,-0.475692,0.415051,-0.198221,-0.166352,0.366860,0.154225
47,0.867850,-0.215618,0.450728,0.340656,0.176948,1.041175,-0.821253,0.722107,-0.969543,-0.037427,...,0.128191,-1.308836,0.628338,-0.942034,-0.204869,0.082956,-0.090193,0.547877,0.330127,-0.300817


In [8]:
import tensorflow as tf
from tensorflow.keras import layers
import os, datetime

model_path = "model/mesh-quality-8gon-small"
BATCH_SIZE = 128
EPOCHS = 5000
INITIAL_LEARNING_RATE = 1e-1

def model_setup():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(22,)),
        layers.BatchNormalization(),

        layers.Dense(16, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(16, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(1),
    ])
    return model

def train_model(model, train_features, train_label, model_epochs=EPOCHS, model_batch_size=BATCH_SIZE, learning_rate=INITIAL_LEARNING_RATE):
    # Callbacks
    # logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    # tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=EPOCHS//5, min_delta=0.0001)

    # Learning rate scheduler
    decay_steps= 10_000
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        learning_rate, 
        decay_steps, 
        1e-2
        )

    # Initialize model
    model.compile(loss=tf.losses.MeanAbsoluteError(),
                optimizer=tf.optimizers.Adam(
                learning_rate=lr_schedule
            ),
            )

    # Train model and write progress to history (for graphing)
    history = model.fit(train_features,
                        train_labels,
                        epochs=model_epochs,
                        batch_size=model_batch_size,
                        validation_split=0.18,
                        verbose=2,
                        callbacks=[checkpoint, early_stopping],
                        )
    return history

model = model_setup()
model.summary()
history = train_model(model, train_features, train_labels)

# ======================
#       EVALUATION
# ======================
train_acc = model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))   